In [ ]:
This notebook is an illustration of Python modules built for GIS analysis.

In [ ]:
import numpy as np
import geog  ## package for geodesic functions
import shapely.geometry
import json
import geopandas as gpd
import overpass # Python package to interface with OSM Overpass API
import folium


In [ ]:

####inputs
radius = 3 # radius in KM
starting_lat = 40.679044
starting_lon = -73.971655


In [ ]:
Create polygon based on input parameters above, and create GeoDataFrame for polygon

In [ ]:

crs = {'init': 'epsg:4326'}
#### create geometry based on a single point
p = shapely.geometry.Point([ starting_lon, starting_lat ])


#### generate polygon based on radius around point
n_points = 20
d = radius * 1000  # meters
angles = np.linspace(0, 360, n_points)
polygon = geog.propagate(p, angles, d)
#geoj = json.loads(json.dumps(shapely.geometry.mapping(shapely.geometry.Polygon(polygon))))  ### code to convert polygon to geoJSON 

# Create GeoDataFrame based on polygon around point
geoj= gpd.GeoDataFrame(crs=crs)
geoj['geometry'] = None
geoj.loc[0, 'geometry'] = shapely.geometry.Polygon(polygon)


In [ ]:


outline_box_osm = ''

for i in [1,0,3,2]:
    outline_box_osm  = outline_box_osm + str(geoj['geometry'][0].bounds[i])+','

outline_box_osm=outline_box_osm[0:-1]
print (outline_box_osm)


with open('data.json', 'w') as outfile:
    json.dump(shapely.geometry.mapping(shapely.geometry.Polygon(polygon)),outfile)


api = overpass.API()
response = api.Get('node[\"amenity\"=\"restaurant\"][\"cuisine\"=\"pizza\"]('+outline_box_osm+')')

##print (response)


poly = gpd.read_file('data.json')
gs = gpd.GeoDataFrame.from_features(response['features'],crs=crs)

print (len(gs))

#print("Head of points data ",gs.head())

pointInPolys = gpd.sjoin(gs,geoj ,op='within')


print (len(pointInPolys))






import matplotlib.pyplot as plt
import mplleaflet

geoj.plot()
pointInPolys.plot()


zoom_level = (radius * -1) + 16




mapa  =  folium.Map([starting_lat, starting_lon ],  zoom_start=zoom_level,tiles='cartodbpositron')


#folium.GeoJson(geoj).add_to(m)
#folium.GeoJson(gs).add_to(m)

style_function = lambda x: {'fillColor': '#0000ff'}


##gj = folium.GeoJson(pointInPolys).add_to(m)

#gj.add_child(folium.Icon(color='green'))
#gj.add_child(folium.Popup('outline Popup on GeoJSON'))
#gj.add_to(m)
#m.save( 'geopandas_1.html')

#print (pointInPolys.head())

from folium import IFrame, Map
from folium.plugins import MarkerCluster


table = """
<!DOCTYPE html>
<html>
<head>
<style>
table {{
    width:100%;
}}
table, th, td {{
    border: 1px solid black;
    border-collapse: collapse;
}}
th, td {{
    padding: 5px;
    text-align: left;
}}
table#t01 tr:nth-child(odd) {{
    background-color: #eee;
}}
table#t01 tr:nth-child(even) {{
   background-color:#fff;
}}
</style>
</head>
<body>

<table id="t01">
  <tr>
    <td>Cuisine Type</td>
    <td>{}</td>
  </tr>
  <tr>
    <td>Restaurant Name</td>
    <td>{}</td>
  </tr>
</table>
</body>
</html>
""".format

import matplotlib
matplotlib.pyplot.close('all')
width, height = 310,110
popups, locations = [], []
for idx, row in pointInPolys.iterrows():
    locations.append([row['geometry'].y, row['geometry'].x])
    name = row['name'].encode('ascii', 'xmlcharrefreplace')

    iframe = IFrame(table('Pizza', name), width=width, height=height)
    popups.append(iframe)

t = folium.FeatureGroup(name='Thermoelectric')
t.add_child(MarkerCluster(locations=locations, popups=popups))
mapa.add_child(t)


folium.GeoJson(geoj).add_to(mapa)
mapa.save('geo2.html')

mapa
